In [ ]:
"""
Purpose: To get the synapses of the
cells that were manually proofread and the
corresponding august ids so that those can be processed

"""

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from os import sys
sys.path.append("/meshAfterParty/meshAfterParty/")
import datajoint as dj
import datajoint_utils as du

minnie,schema = du.configure_minnie_vm()
m65mat = du.m65mat

INFO - 2021-05-19 15:34:39,608 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 15:34:39,609 - settings - Setting database.user to celiib
INFO - 2021-05-19 15:34:39,610 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 15:34:39,614 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 15:34:40,312 - settings - Setting enable_python_native_blobs to True


In [5]:
current_version = 88
import allen_utils as alu
from annotationframeworkclient import FrameworkClient
client = alu.configure_client(current_version)

In [6]:
import pandas as pd
import pandas_utils as pu
import numpy as np

# Pulling Down the Table with the current proof status

In [7]:
"""
The proofread segments were not stored anywhere in the 
database but instead were on the spreadsheet 
szi-chieh shared with  me:

https://docs.google.com/spreadsheets/d/1lbLtQegeG7UPzGOmYE-lliPqTBldC9BP-JSzrWi8Ybo/edit?pli=1#gid=0

"""


'\nThe proofread segments were not stored anywhere in the \ndatabase but instead were on the spreadsheet \nszi-chieh shared with  me:\n\nhttps://docs.google.com/spreadsheets/d/1lbLtQegeG7UPzGOmYE-lliPqTBldC9BP-JSzrWi8Ybo/edit?pli=1#gid=0\n\n'

In [8]:
man_proof_df = alu.manual_proof_csv_to_annotated_df(csv_path='./Minnie65 core proofreading_5_3.csv',
                                    partial_strategy_idx_start = 83,
                                    partial_strategy_idx_end = 159)
man_proof_df

/meshAfterParty/meshAfterParty/allen_utils.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proof_pd.iloc[partial_strategy_idx_start+1:partial_strategy_idx_end]["extension_type"]  = "partial"


,Nucleus ID,Finished_Den,Finished_Ax,extension_type
1,296726.0,True,True,full
2,294545.0,True,True,full
4,225498.0,True,True,full
5,335175.0,True,True,full
6,258307.0,True,True,full
...,...,...,...,...
156,580774.0,True,True,partial
158,582091.0,True,True,partial
161,259167.0,True,True,full
164,189149.0,True,True,full


In [9]:
man_proof_nuc_ids = man_proof_df["Nucleus ID"].to_numpy()
man_proof_nuc_ids

array([296726., 294545., 225498., 335175., 258307., 271518., 269380.,
       331945., 300763., 230236., 264870., 327859., 262773., 330079.,
       301095., 301189., 330326., 269247., 332199., 236197., 559381.,
       533187., 564808., 489675., 493419., 525758., 493968., 518898.,
       556823., 531202., 522656., 518848., 487512., 460053., 557121.,
       557030., 525498., 559081., 460391., 493885., 493806., 520364.,
       525405., 588983., 560732., 588839., 560217., 560109., 562808.,
       582129., 553325., 582056., 554921., 610498., 554741., 583848.,
       583961., 582210., 560530., 585723., 558709., 526226., 554200.,
       518853., 558684., 516758., 517056., 554833., 581967., 524491.,
       587426., 616159., 553283., 554891., 580826., 612143., 612352.,
       518623., 520027., 553321., 553407., 580988., 583891., 516621.,
       518224., 580948., 612266., 518004., 520182., 581998., 583729.,
       583739., 551802., 553339., 610396., 610607., 610403., 610535.,
       582313., 5824

In [10]:
nuc_table = du.configure_nucleus_table(version = current_version)
man_proof_info = (nuc_table & [dict(nucleus_id=k) for k in man_proof_nuc_ids]).proj("nucleus_id","segment_id").fetch(as_dict=True)
man_proof_info_df = pd.DataFrame.from_dict(man_proof_info)
man_proof_info_df

,ver,nucleus_id,segment_id
0,88.00,189149,864691135855890478
1,88.00,223037,864691136227167569
2,88.00,225498,864691135753932237
3,88.00,230236,864691135609687047
4,88.00,236197,864691136903144370
...,...,...,...
136,88.00,610615,864691135367284729
137,88.00,612143,864691135341198789
138,88.00,612266,864691135739733140
139,88.00,612352,864691136194400332


In [ ]:
segment_id = 864691135609687047
alu.segment_id_to_pre_post_syn_neuroglancer(segment_id)

# Comparing the Segments Against those Used for Training

In [11]:
man_proof_seg_ids = man_proof_info_df["segment_id"].to_numpy()

In [12]:
training_seg_id = minnie.AutoProofreadValidationSegmentMap.fetch("segment_id")
training_seg_id.shape
np.intersect1d(man_proof_seg_ids,training_seg_id).shape

(52,)

# Creating the Spreadsheets for Validation

In [13]:
from pathlib import Path
old_spreadsheets_folder = Path("./Syanpse_dfs/")
old_spread_files = list(old_spreadsheets_folder.iterdir())
len(old_spread_files)

69

In [14]:
nucleus_table = du.configure_nucleus_table(current_version)
nucleus_id,seg_id_new = nucleus_table.fetch("nucleus_id","segment_id")

In [15]:
df = pd.DataFrame()
df["nucleus_id"] = nucleus_id
df["segment_id_88"] = seg_id_new
unique_ids, counts = np.unique(seg_id_new,return_counts=True)
multi_nuclei = unique_ids[counts>1]
df_man = df[(df["segment_id_88"].isin(man_proof_seg_ids)) & 
           ~(df["segment_id_88"].isin(multi_nuclei))]
df_man

,nucleus_id,segment_id_88
24557,189149,864691135855890478
32642,223037,864691136227167569
33713,225498,864691135753932237
35424,230236,864691135609687047
37176,236197,864691136903144370
...,...,...
123896,610615,864691135367284729
124397,612143,864691135341198789
124437,612266,864691135739733140
124474,612352,864691136194400332


# Getting the Old Segment Ids that correspond to this segment id

In [16]:
import system_utils as su

In [17]:
aug_df = alu.get_aug_nuc_df()[["flat_nuc_id","root_id"]]
old_new_map = pd.merge(df_man,aug_df,left_on="nucleus_id",right_on="flat_nuc_id",how='inner')
old_new_map

,nucleus_id,segment_id_88,flat_nuc_id,root_id
0,189149,864691135855890478,189149,864691134965302020
1,223037,864691136227167569,223037,864691135538120050
2,225498,864691135753932237,225498,864691135564655959
3,230236,864691135609687047,230236,864691135748568361
4,236197,864691136903144370,236197,864691134753831925
...,...,...,...,...
135,610615,864691135367284729,610615,864691136664412413
136,612143,864691135341198789,612143,864691136050853107
137,612266,864691135739733140,612266,864691136618438541
138,612352,864691136194400332,612352,864691135012395766


# Creating the Tables Used For Validation

In [18]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-05-19 15:35:47,069 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 15:35:47,071 - settings - Setting database.user to celiib
INFO - 2021-05-19 15:35:47,072 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 15:35:47,078 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 15:35:47,869 - settings - Setting enable_python_native_blobs to True


In [259]:
mapping_cols = ["nucleus_id", "segment_id",
                  "old_segment_id",
                  "finished_dendrite",
                  "finished_axon",
                  "extension_type"]
old_new_map_with_labels = pd.merge(old_new_map,man_proof_df,left_on="nucleus_id",right_on="Nucleus ID",how='inner')
old_new_map_rev = pu.rename_columns(old_new_map_with_labels,
                                    dict(segment_id_88="segment_id",
                                        root_id="old_segment_id",
                                        Finished_Den="finished_dendrite",
                                       Finished_Ax = "finished_axon"))[mapping_cols]

"""
Getting rid of those segments that have a duplicate old segment id (aka it would require a split)
"""
old_segs,old_segs_counts = np.unique(old_new_map_rev["old_segment_id"].to_numpy(),return_counts=True)
repeated_segs = old_segs[old_segs_counts>1]
old_new_map_rev = old_new_map_rev[~old_new_map_rev["old_segment_id"].isin(repeated_segs)]
old_new_map_rev

,nucleus_id,segment_id,old_segment_id,finished_dendrite,finished_axon,extension_type
0,189149,864691135855890478,864691134965302020,True,True,full
1,223037,864691136227167569,864691135538120050,True,True,full
2,225498,864691135753932237,864691135564655959,True,True,full
3,230236,864691135609687047,864691135748568361,True,True,full
4,236197,864691136903144370,864691134753831925,True,True,full
...,...,...,...,...,...,...
135,610615,864691135367284729,864691136664412413,True,True,partial
136,612143,864691135341198789,864691136050853107,True,True,partial
137,612266,864691135739733140,864691136618438541,True,True,partial
138,612352,864691136194400332,864691135012395766,True,True,partial


# Checking the Old Segment Ids are Bigger than just the nucleus mesh (if not then manually changing them)

In [260]:
# original_mesh_sizes = []
# new_seg_ids = []
# from tqdm_utils import tqdm
# old_segs = old_new_map_rev["old_segment_id"].to_numpy()
# for k in tqdm(old_segs):
#     original_mesh_sizes.append((minnie.Decimation() & dict(segment_id=k,
#                                                          decimation_ratio=0.25)).fetch1("n_faces"))
#     new_seg_ids.append(vu.old_to_new_segment_id(k))

# original_mesh_sizes = np.array(original_mesh_sizes)
# new_seg_ids = np.array(new_seg_ids)

# # noticed that there was a distinct barrier from 30k faces to 300k on decimation
# sorted_indices = np.argsort(original_mesh_sizes)
# print(old_segs[sorted_indices][0:8])
# print(new_seg_ids[sorted_indices][0:8])


# # checking each segment
# segment_id = 864691134602307367
# DiX = alu.seg_id_to_nx_lineage_graph(segment_id,return_graph_endnodes=False)

# DiX[segment_id]

In [261]:
new_mapping = {864691134753831925:864691135758569550,
              864691134965302020:864691135585319036,
               864691136697492221:864691135099943968,
              864691136016130227:864691135058593947,
               864691135395928454:864691136273691661,
               864691134640069900:864691135467605132,
               864691135733588629:864691136451113215,
               864691134602307367:864691136672696199
              }
#mapping the old segments to the new segments
old_segs = old_new_map_rev["old_segment_id"].to_numpy()
old_segs_mapped = [new_mapping[k] if k in new_mapping.keys() else k for
                   k in old_segs]
old_new_map_rev_cleaned = old_new_map_rev.copy()
old_new_map_rev_cleaned["old_segment_id"] = old_segs_mapped
old_new_map_rev_cleaned

# #checked the substitute was good
# import numpy as np
# np.intersect1d(old_new_map_rev_cleaned["old_segment_id"].to_numpy())

,nucleus_id,segment_id,old_segment_id,finished_dendrite,finished_axon,extension_type
0,189149,864691135855890478,864691135585319036,True,True,full
1,223037,864691136227167569,864691135538120050,True,True,full
2,225498,864691135753932237,864691135564655959,True,True,full
3,230236,864691135609687047,864691135748568361,True,True,full
4,236197,864691136903144370,864691135758569550,True,True,full
...,...,...,...,...,...,...
135,610615,864691135367284729,864691136664412413,True,True,partial
136,612143,864691135341198789,864691136050853107,True,True,partial
137,612266,864691135739733140,864691136618438541,True,True,partial
138,612352,864691136194400332,864691135012395766,True,True,partial


# Documenting Properties of Raw Neurons

In [256]:
"""
Properties need to keep track of: 
1) Large splits
2) n_somas
3) n_glia
4) soma_mergers
"""

import pandas_utils as pu
import pandas as pd
df = pu.csv_to_df('./v4_validation_neuron_features.csv') 
df


,id,n_somas,n_soma_mergers,n_glia,large_splits,notes
0,864691135585319036,1,0,0,0,NaN
1,864691135538120050,2,0,0,0,NaN
2,864691135564655959,1,0,0,0,NaN
3,864691135748568361,1,0,0,0,NaN
4,864691135758569550,2,0,2,0,NaN
...,...,...,...,...,...,...
133,864691136664412413,2,0,0,0,half soma merge
134,864691136050853107,1,0,0,0,NaN
135,864691136618438541,1,0,0,0,NaN
136,864691135012395766,3,0,0,0,third soma is very small


In [263]:
old_new_map_annotated = pd.merge(old_new_map_rev_cleaned,df,left_on="old_segment_id",right_on="id")[mapping_cols + 
                                                                                            ["n_somas",
                                                                                           "n_soma_mergers",
                                                                                           "n_glia",
                                                                                           "large_splits",
                                                                                           "notes"]]
old_new_map_annotated

,nucleus_id,segment_id,old_segment_id,finished_dendrite,finished_axon,extension_type,n_somas,n_soma_mergers,n_glia,large_splits,notes
0,189149,864691135855890478,864691135585319036,True,True,full,1,0,0,0,NaN
1,223037,864691136227167569,864691135538120050,True,True,full,2,0,0,0,NaN
2,225498,864691135753932237,864691135564655959,True,True,full,1,0,0,0,NaN
3,230236,864691135609687047,864691135748568361,True,True,full,1,0,0,0,NaN
4,236197,864691136903144370,864691135758569550,True,True,full,2,0,2,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
133,610615,864691135367284729,864691136664412413,True,True,partial,2,0,0,0,half soma merge
134,612143,864691135341198789,864691136050853107,True,True,partial,1,0,0,0,NaN
135,612266,864691135739733140,864691136618438541,True,True,partial,1,0,0,0,NaN
136,612352,864691136194400332,864691135012395766,True,True,partial,3,0,0,0,third soma is very small


# Writing Information to Table

In [273]:
dict_to_write = [dict(k,
                  ver=ver,
                 old_ver=old_ver) for k in pu.dataframe_to_row_dicts(old_new_map_annotated)]
dict_to_write

[{'nucleus_id': 189149,
  'segment_id': 864691135855890478,
  'old_segment_id': 864691135585319036,
  'finished_dendrite': 'True',
  'finished_axon': 'True',
  'extension_type': 'full',
  'n_somas': 1,
  'n_soma_mergers': 0,
  'n_glia': 0,
  'large_splits': 0,
  'notes': nan,
  'ver': 88,
  'old_ver': 0.08},
 {'nucleus_id': 223037,
  'segment_id': 864691136227167569,
  'old_segment_id': 864691135538120050,
  'finished_dendrite': 'True',
  'finished_axon': 'True',
  'extension_type': 'full',
  'n_somas': 2,
  'n_soma_mergers': 0,
  'n_glia': 0,
  'large_splits': 0,
  'notes': nan,
  'ver': 88,
  'old_ver': 0.08},
 {'nucleus_id': 225498,
  'segment_id': 864691135753932237,
  'old_segment_id': 864691135564655959,
  'finished_dendrite': 'True',
  'finished_axon': 'True',
  'extension_type': 'full',
  'n_somas': 1,
  'n_soma_mergers': 0,
  'n_glia': 0,
  'large_splits': 0,
  'notes': nan,
  'ver': 88,
  'old_ver': 0.08},
 {'nucleus_id': 230236,
  'segment_id': 864691135609687047,
  'old_seg

In [276]:
AutoProofreadValidationSegmentMap4.drop()

`microns_minnie65_02`.`auto_proofread_validation_segment_map4` (0 tuples)
Proceed? [yes, No]: yes


INFO - 2021-05-21 20:07:44,642 - table - Dropped table `microns_minnie65_02`.`auto_proofread_validation_segment_map4`


Tables dropped.  Restart kernel.


In [284]:
ex_dict = dict_to_write[0]
ex_dict["notes"] = ""
ex_dict

{'nucleus_id': 189149,
 'segment_id': 864691135855890478,
 'old_segment_id': 864691135585319036,
 'finished_dendrite': 'True',
 'finished_axon': 'True',
 'extension_type': 'full',
 'n_somas': 1,
 'n_soma_mergers': 0,
 'n_glia': 0,
 'large_splits': 0,
 'notes': '',
 'ver': 88,
 'old_ver': 0.08}

In [285]:
AutoProofreadValidationSegmentMap4.insert1(ex_dict)

In [287]:
#cell_type_predicted: enum('full','partial')

@schema
class AutoProofreadValidationSegmentMap4(dj.Manual):
    definition="""
    -> m65mat.Materialization
    nucleus_id           : bigint unsigned              # nucleus id
    segment_id           : bigint unsigned              # the segment id in the proofread version
    -> m65mat.Materialization.proj(old_ver="ver")
    ---
    old_segment_id       : bigint unsigned              # the segment id in the proofread version--
    finished_dendrite    : enum('True','False')         # whether axon was finished cleaning
    finished_axon        : enum('True','False')         #whether axon was finished cleaning
    extension_type       : enum('full','partial')
    n_somas              : tinyint unsigned
    n_soma_mergers       : tinyint unsigned
    n_glia               : tinyint unsigned
    large_splits         : tinyint unsigned
    notes=NULL           : varchar(256) 
    """
old_ver = 0.08
ver = 88

dict_to_write = []
for k in pu.dataframe_to_row_dicts(old_new_map_annotated):
    curr_dict = dict(k,
                  ver=ver,
                 old_ver=old_ver)
    try:
        if np.isnan(curr_dict["notes"]):
            curr_dict["notes"] = ""
    except:
        pass
    
    dict_to_write.append(curr_dict)

AutoProofreadValidationSegmentMap4.insert(dict_to_write,skip_duplicates=True)
AutoProofreadValidationSegmentMap4()

ver materialization version,nucleus_id nucleus id,segment_id the segment id in the proofread version,old_ver materialization version,old_segment_id the segment id in the proofread version--,finished_dendrite whether axon was finished cleaning,finished_axon whether axon was finished cleaning,extension_type,n_somas,n_soma_mergers,n_glia,large_splits,notes
88.00,189149,864691135855890478,0.08,864691135585319036,True,True,full,1,0,0,0,
88.00,223037,864691136227167569,0.08,864691135538120050,True,True,full,2,0,0,0,
88.00,225498,864691135753932237,0.08,864691135564655959,True,True,full,1,0,0,0,
88.00,230236,864691135609687047,0.08,864691135748568361,True,True,full,1,0,0,0,
88.00,236197,864691136903144370,0.08,864691135758569550,True,True,full,2,0,2,0,
88.00,258307,864691135293126156,0.08,864691136194042326,True,True,full,1,0,0,0,
88.00,259167,864691134988722810,0.08,864691135700248187,True,True,full,2,0,0,0,
88.00,262773,864691136023889209,0.08,864691135233108569,True,True,full,1,0,0,0,
88.00,264870,864691136951664863,0.08,864691135740225387,True,True,full,1,0,0,0,
88.00,269247,864691135584074360,0.08,864691136105498585,True,True,full,1,0,0,0,


In [21]:
"""
Represents the older segments that need to be pulled down
"""

@schema
class AutoProofreadValidationSegment4(dj.Manual):
    definition="""
    segment_id           : bigint unsigned              # the segment id in the proofread version
    """

old_segment_id = AutoProofreadValidationSegmentMap4.fetch("old_segment_id")
AutoProofreadValidationSegment4.insert([dict(segment_id=k) for k in old_segment_id],skip_duplicates=True)
AutoProofreadValidationSegment4()

segment_id the segment id in the proofread version
864691134602307367
864691134640069900
864691134753831925
864691134965302020
864691134988385914
864691134988386682
864691135012395766
864691135012541942
864691135058565275
864691135065011524


In [74]:
"""
Segments that should not be included: 
"""
@schema
class AutoProofreadValidationSegmentExclude4(dj.Manual):
    definition="""
    segment_id           : bigint unsigned              # the segment id in the proofread version
    ---
    criteria             : varchar(256)                 # why the segments were excluded
    """
    
nuclei_meshes = np.array([864691134753831925, 864691134965302020, 864691136016130227,
       864691136697492221])

exclude_meshes = {
}
reason = "only nuclei meshes"
AutoProofreadValidationSegmentExclude4.insert([dict(segment_id=k,
                                                   criteria=reason) for k in nuclei_meshes],
                                             skip_duplicates=True)
AutoProofreadValidationSegmentExclude4()

segment_id the segment id in the proofread version,criteria why the segments were excluded
864691134753831925,only nuclei meshes
864691134965302020,only nuclei meshes
864691136016130227,only nuclei meshes
864691136697492221,only nuclei meshes


# Creating the Spreadsheets of the Synapses

In [ ]:
# where the synapses for the new segments are kept
minnie.AutoProofreadValidationManualSynapseUpdated() 

In [ ]:
# where the synapses for the old segments are kept
minnie.AutoProofreadValidationNucleusSynapse()

In [ ]:
AutoProofreadValidationSegmentMap4.fetch("segment_id")

# Updateing the Manual Proofread Final Synapses

In [25]:
"""
Purpose: To Create a table that has the 
valid synpases for the proofread targets
that can be used to restrict the set determined in the synapse spreadsheet

Pseudocode: 

"""

@schema
class AutoProofreadValidationManualSynapse4(dj.Manual):
    definition="""
    -> m65mat.Materialization
    synapse_id           : bigint unsigned              # synapse index within the segmentation
    presyn               : bigint unsigned              # segment_id of the presynaptic cell. Equivalent to Allen 'pt_root_id'.
    postsyn              : bigint unsigned              # segment_id of the postsynaptic cell. Equivalent to Allen 'pt_root_id'.
    ---
    synapse_x            : int unsigned                 # x coordinate of synapse centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.
    synapse_y            : int unsigned                 # y coordinate of centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.
    synapse_z            : int unsigned                 # z coordinate of centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.
    synapse_size         : int unsigned                 # (EM voxels) scaled by (4x4x40)
    """

In [ ]:
# # import datajoint as dj
# # (dj.U("ver") & du.m65mat.ProofreadSynapse()).fetch("ver")

# # Example Segment
# segment_id = 864691135367284729
# du.m65mat.ProofreadSynapse() & f"presyn={segment_id} OR postsyn={segment_id}" & "ver=88"

In [26]:
postsyn_table = du.m65mat.ProofreadSynapse & AutoProofreadValidationSegmentMap4.proj(postsyn='segment_id') & f"ver={current_version}"
postsyn_data = postsyn_table.proj("synapse_x","synapse_y","synapse_z","synapse_size").fetch(as_dict=True)
print(f"len(postsyn_data) = {len(postsyn_data)}")
AutoProofreadValidationManualSynapse4.insert(postsyn_data,skip_duplicates=True)

In [28]:
presyn_table = du.m65mat.ProofreadSynapse & AutoProofreadValidationSegmentMap4.proj(presyn='segment_id') & f"ver={current_version}"
presyn_data = presyn_table.proj("synapse_x","synapse_y","synapse_z","synapse_size").fetch(as_dict=True)
print(f"len(presyn_data) = {len(presyn_data)}")
AutoProofreadValidationManualSynapse4.insert(presyn_data,skip_duplicates=True)

In [32]:
AutoProofreadValidationManualSynapse4()

ver materialization version,synapse_id synapse index within the segmentation,presyn segment_id of the presynaptic cell. Equivalent to Allen 'pt_root_id'.,postsyn segment_id of the postsynaptic cell. Equivalent to Allen 'pt_root_id'.,"synapse_x x coordinate of synapse centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.","synapse_y y coordinate of centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.","synapse_z z coordinate of centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm). From Allen 'ctr_pt_position'.",synapse_size (EM voxels) scaled by (4x4x40)
88.00,1259344,864691135584074360,864691134359870879,70498,205776,19771,4364
88.00,4305885,864691135584074360,864691135759542863,73084,208952,19833,5980
88.00,4305971,864691135584074360,864691135614224678,73183,209389,19799,11208
88.00,4306209,864691135584074360,864691136098441186,73478,209356,19817,1520
88.00,4306873,864691135584074360,864691135783254863,74366,209722,19867,15220
88.00,4307545,864691135584074360,864691135169715403,75164,211342,19880,13888
88.00,4307820,864691135584074360,864691136226950737,75464,211714,19846,3152
88.00,4308077,864691135584074360,864691135941133940,75936,211552,19899,6028
88.00,5308430,864691135584074360,864691135647095590,72443,208132,19823,8240
88.00,6746551,864691136903228594,864691136436641694,78076,108033,18759,18228


In [36]:
# Testing out the synapses were inserted correctl
from importlib import reload
import validation_utils as vu
minnie,schema = du.configure_minnie_vm()
du = reload(du)
vu = reload(vu)

segment_id = 864691135367284729
vu.updated_proofread_postsyn_ids(segment_id).shape,vu.updated_proofread_presyn_ids(segment_id).shape

INFO - 2021-05-19 15:46:48,241 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 15:46:48,243 - settings - Setting database.user to celiib
INFO - 2021-05-19 15:46:48,244 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 15:46:48,248 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 15:46:49,158 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 15:46:49,165 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 15:46:49,166 - settings - Setting database.user to celiib
INFO - 2021-05-19 15:46:49,166 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 15:46:49,167 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 15:46:49,168 - settings - Setting database.user to celiib
INFO - 2021-05-19 15:46:49,169 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 15:46:49,169 - settings - Setting enable_pytho

((3302,), (176,))

# Getting the Original Synapses

In [60]:
alu = reload(alu)
alu.client.materialize._version

89

In [82]:
original_seg_ids = ((minnie.Decimation() & minnie.AutoProofreadValidationSegment4() & "decimation_ratio=0.25") 
                    - minnie.AutoProofreadValidationSegmentExclude4()).fetch("segment_id")
original_seg_ids.shape

(134,)

In [177]:
vu.old_segment_id_validation_synapses(segment_id = original_seg_ids[2],
verbose = True,
plot_original_mesh = True,
plot_potential_synapses = True,
plot_matched_synapses = True)

INFO - 2021-05-20 21:15:49,720 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-20 21:15:49,723 - settings - Setting database.user to celiib
INFO - 2021-05-20 21:15:49,725 - settings - Setting database.password to newceliipass
INFO - 2021-05-20 21:15:49,729 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-20 21:15:51,221 - settings - Setting enable_python_native_blobs to True


# of end nodes = 25
last_datetime_edited = 2021-01-29 07:24:32.323000
# of presyn = 286, # of postsyn = 4848
Total synapses BEFORE filtering = 5134
Total synapses AFTER filtering = 5090


/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


Potential Synapses


3814 out of 5090 were within threshold of original mesh
Potential Synapses


/meshAfterParty/meshAfterParty/validation_utils.py:2431: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  syn_match["synapse_x"] = np.array(list(syn_match["ctr_pt_position"].to_numpy()))[:,0]
/meshAfterParty/meshAfterParty/validation_utils.py:2432: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  syn_match["synapse_y"] = np.array(list(syn_match["ctr_pt_position"].to_numpy()))[:,1]
/meshAfterParty/meshAfterParty/validation_utils.py:2433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

NameError: name 'syanspe_df' is not defined

# Notes for Implementation

In [ ]:
"""
Things to worry about: 
1) Double Soma



"""

# Functions Used in Proofreading

In [ ]:
updated_proofread_synapse_table